In [ ]:
### ML ASSIGNMENT

In [ ]:
### Get location

In [2]:
%%time
# import libraries

import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role

role = get_execution_role()
prefix = 'house_price_xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the eu-west-1 region. You will use the 685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.
CPU times: user 1.82 s, sys: 182 ms, total: 2 s
Wall time: 11.7 s


In [ ]:
### create bucket - if not allready created?

In [3]:
%%time
bucket_name = 'mutest-01' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
      print("create bucket")
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
      print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.
CPU times: user 76.9 ms, sys: 8.71 ms, total: 85.7 ms
Wall time: 519 ms


In [144]:
my_region

'eu-west-1'

NameError: name 'client' is not defined

In [4]:
#### CHECK contents of bucket #####
##from boto3 import client
##conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
##for key in conn.list_objects(Bucket='mutest-01')['Contents']:
##    print(key['Key'])
try:
    conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
    for key in conn.list_objects(Bucket='mutest-01')['Contents']:
        print(key['Key'])
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)    

S3 error:  name 'client' is not defined


In [ ]:
### download data and load data into dataframe

In [145]:
%%time
##### Experimenting with data sources
####urlX ='http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv'
####url1 = 'http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018-part1.csv'
####url2 = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018-part2.csv'
####url3 = 'http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv'

###### EASY TEST SOURCE - but not fully populated
url = 'http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-monthly-update-new-version.csv'

### Load individually to make it easier
###### 2019 - complete
url4 = 'http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv'
###### 2018 part 2
url5 = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018-part2.csv'
###### 2018 part 1
url6 = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018-part1.csv'
#####  2017 - part 2
url7 = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2017-part2.csv'
#### 2017  - part 1
url8 = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2017-part1.csv'

try:
    model_data = pd.read_csv(url,delimiter=',')   ### incomplete source
    model_data4 = pd.read_csv(url4,delimiter=',') ### 2019
    model_data5 = pd.read_csv(url5,delimiter=',') ### 2018 - part 2
    model_data6 = pd.read_csv(url6,delimiter=',') ### 2018 - part 1
    model_data7 = pd.read_csv(url7,delimiter=',') ### 2017 - part 2
    model_data8 = pd.read_csv(url8,delimiter=',') ### 2017 - part 1
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.
CPU times: user 12.7 s, sys: 623 ms, total: 13.3 s
Wall time: 18.3 s


In [146]:
### have a look at the data
model_data.head(5)

,{8F1B26BD-4F82-53DB-E053-6C04A8C03649},289000,2006-08-25 00:00,SW10 0PR,F,N,L,GROUND FLOOR FLAT 11,Unnamed: 8,BURNABY STREET,Unnamed: 10,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A.1
0,{8F1B26BD-9496-53DB-E053-6C04A8C03649},375000,2006-08-21 00:00,BN24 6EH,D,N,F,HOOK HOUSE,NaN,SEA ROAD,PEVENSEY BAY,PEVENSEY,WEALDEN,EAST SUSSEX,A,A
1,{8F1B26BD-A900-53DB-E053-6C04A8C03649},445000,2006-01-12 00:00,GL55 6DZ,T,N,F,BLAKEMANS COTTAGE,NaN,LOWER HIGH STREET,NaN,CHIPPING CAMPDEN,COTSWOLD,GLOUCESTERSHIRE,A,A
2,{8F1B26BE-191A-53DB-E053-6C04A8C03649},121950,2006-06-30 00:00,WS11 9NW,F,Y,L,2,NaN,ST JAMES PLACE,NORTON CANES,CANNOCK,CANNOCK CHASE,STAFFORDSHIRE,A,A
3,{8F1B26BE-1E7B-53DB-E053-6C04A8C03649},161000,2006-01-20 00:00,BR5 2DD,F,N,F,10A,NaN,MARION CRESCENT,NaN,ORPINGTON,BROMLEY,GREATER LONDON,A,A
4,{8F1B26BE-279A-53DB-E053-6C04A8C03649},321111,2006-12-20 00:00,SY8 1DJ,S,N,F,112,NaN,CORVE STREET,NaN,LUDLOW,SHROPSHIRE,SHROPSHIRE,A,A


In [11]:

### remove unwanted cols
cols = [0,3,5,7,8,9,10,11,12,14,15] ### remove unwanted cols

def drop_cols(fname):
    fname.drop(fname.columns[cols],axis=1,inplace=True)

drop_cols(model_data)
drop_cols(model_data4)
drop_cols(model_data5)
drop_cols(model_data6)
drop_cols(model_data7)
drop_cols(model_data8)

CPU times: user 248 ms, sys: 41.9 ms, total: 290 ms
Wall time: 289 ms


In [12]:
model_data.head(1)

,289000,2006-08-25 00:00,F,L,GREATER LONDON
0,375000,2006-08-21 00:00,D,F,EAST SUSSEX


In [13]:
### rename cols

def rename_cols(data_source):
    data_source.rename(columns={ data_source.columns[0]: "price", data_source.columns[1]: "date", data_source.columns[2]: "type",data_source.columns[3]: "lease", data_source.columns[4]: "location" }, inplace = True)

rename_cols(model_data)
rename_cols(model_data4)
rename_cols(model_data5)
rename_cols(model_data6)
rename_cols(model_data7)
rename_cols(model_data8)

In [14]:
model_data.head(1)

,price,date,type,lease,location
0,375000,2006-08-21 00:00,D,F,EAST SUSSEX


In [ ]:
%%time
### ONE HOT ENCODING

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def one_hot_encoder(model_dataX):
    h_type = LabelEncoder()
    h_lease = LabelEncoder()

    model_dataX['type_encoded'] = h_type.fit_transform(model_dataX.type)
    model_dataX['lease_encoded'] = h_lease.fit_transform(model_dataX.lease)

    type_ohe = OneHotEncoder()
    lease_ohe = OneHotEncoder()

    X = type_ohe.fit_transform(model_dataX.type_encoded.values.reshape(-1,1)).toarray()
    Xm = lease_ohe.fit_transform(model_dataX.lease_encoded.values.reshape(-1,1)).toarray()

    dfOneHot = pd.DataFrame(X, columns = ["type_"+str(int(i)) for i in range(X.shape[1])])
    model_dataX = pd.concat([model_dataX, dfOneHot], axis=1)

    dfOneHot = pd.DataFrame(Xm, columns = ["lease_"+str(int(i)) for i in range(Xm.shape[1])])
    ###model_data = pd.concat([model_data, dfOneHot], axis=1)
    return model_dataX

model_data = one_hot_encoder(model_data)

model_data4 = one_hot_encoder(model_data4)

model_data5 =  one_hot_encoder(model_data5)

model_data6 = one_hot_encoder(model_data6)

model_data7 = one_hot_encoder(model_data7)

model_data8 = one_hot_encoder(model_data8)  


In [19]:
model_data7.head(1)

,price,date,type,lease,location_london,type_encoded,lease_encoded,type_D,type_F,type_O,type_S,type_T
0,185000,2017,S,F,RUTLAND,3,0,0.0,0.0,0.0,1.0,0.0


In [17]:
%%time
### rename one hot encoded cols and convert all to ints
def convert_to_int(m):
    m.rename(columns={ m.columns[4]: "location_london", m.columns[7]: "type_D", m.columns[8]: "type_F", m.columns[9]: "type_O",m.columns[10]: "type_S", m.columns[11]: "type_T" }, inplace = True)
    ### date to year to int
    m["date"] = pd.DatetimeIndex(m["date"]).year.astype(int)
    ### convert to int

convert_to_int(model_data)
convert_to_int(model_data4)
convert_to_int(model_data5)
convert_to_int(model_data6)
convert_to_int(model_data7)
convert_to_int(model_data8)    

CPU times: user 2.23 s, sys: 221 ms, total: 2.45 s
Wall time: 1.26 s


In [134]:
model_data.head(5)

,predicted,new-col,price,date,location_london,lease_encoded,type_D,type_F,type_O,type_S,type_T
50487,93791,93791,571000,2019,0,0,1.0,0.0,0.0,0.0,0.0
67324,231464,231464,135000,2019,0,0,0.0,0.0,0.0,1.0,0.0
67322,231464,231464,183000,2019,0,0,0.0,0.0,0.0,1.0,0.0
67321,231464,231464,190000,2019,0,0,0.0,0.0,0.0,1.0,0.0
67320,393791,393791,375000,2019,0,0,1.0,0.0,0.0,0.0,0.0


In [20]:
%%time
### location in or out of London - LONDON = 1 NOT LONDON = 0

def in_out(inout):
    inout['location_london'] = inout['location_london'].apply(lambda x: 1 if x == 'GREATER LONDON' else 0)

in_out(model_data)
in_out(model_data4)
in_out(model_data5)
in_out(model_data6)
in_out(model_data7)
in_out(model_data8) 

CPU times: user 1.75 s, sys: 25.7 ms, total: 1.78 s
Wall time: 990 ms


In [21]:
model_data.head(1)

,price,date,type,lease,location_london,type_encoded,lease_encoded,type_D,type_F,type_O,type_S,type_T
0,375000,2006,D,F,0,0,0,1.0,0.0,0.0,0.0,0.0


In [22]:
%%time
model_data.groupby('date').count() ### this file has few data items although good year coverage

CPU times: user 42.7 ms, sys: 8.22 ms, total: 51 ms
Wall time: 33.8 ms


,price,type,lease,location_london,type_encoded,lease_encoded,type_D,type_F,type_O,type_S,type_T
date,,,,,,,,,,,
1995,39,39,39,39,39,39,39,39,39,39,39
1996,65,65,65,65,65,65,65,65,65,65,65
1997,74,74,74,74,74,74,74,74,74,74,74
1998,65,65,65,65,65,65,65,65,65,65,65
1999,65,65,65,65,65,65,65,65,65,65,65
2000,64,64,64,64,64,64,64,64,64,64,64
2001,73,73,73,73,73,73,73,73,73,73,73
2002,116,116,116,116,116,116,116,116,116,116,116
2003,82,82,82,82,82,82,82,82,82,82,82


In [23]:
%%time
### delete not used cols and set up
def dropcols(xdrop):
    xdrop = xdrop.drop(xdrop.columns[[2,3,5]], axis=1)
    xdrop['price'] = round(xdrop['price'],-3) #### £000's
    xdrop = xdrop.sort_values(by=["date"], ascending=False)
    return(xdrop)

model_data = dropcols(model_data)
model_data4 = dropcols(model_data4)
model_data5 = dropcols(model_data5)
model_data6 = dropcols(model_data6)
model_data7 = dropcols(model_data7)
model_data8 = dropcols(model_data8) 

CPU times: user 490 ms, sys: 27.5 ms, total: 517 ms
Wall time: 274 ms


In [141]:
model_data.head(5)

,{8F1B26BD-4F82-53DB-E053-6C04A8C03649},289000,2006-08-25 00:00,SW10 0PR,F,N,L,GROUND FLOOR FLAT 11,Unnamed: 8,BURNABY STREET,Unnamed: 10,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A.1
0,{8F1B26BD-9496-53DB-E053-6C04A8C03649},375000,2006-08-21 00:00,BN24 6EH,D,N,F,HOOK HOUSE,NaN,SEA ROAD,PEVENSEY BAY,PEVENSEY,WEALDEN,EAST SUSSEX,A,A
1,{8F1B26BD-A900-53DB-E053-6C04A8C03649},445000,2006-01-12 00:00,GL55 6DZ,T,N,F,BLAKEMANS COTTAGE,NaN,LOWER HIGH STREET,NaN,CHIPPING CAMPDEN,COTSWOLD,GLOUCESTERSHIRE,A,A
2,{8F1B26BE-191A-53DB-E053-6C04A8C03649},121950,2006-06-30 00:00,WS11 9NW,F,Y,L,2,NaN,ST JAMES PLACE,NORTON CANES,CANNOCK,CANNOCK CHASE,STAFFORDSHIRE,A,A
3,{8F1B26BE-1E7B-53DB-E053-6C04A8C03649},161000,2006-01-20 00:00,BR5 2DD,F,N,F,10A,NaN,MARION CRESCENT,NaN,ORPINGTON,BROMLEY,GREATER LONDON,A,A
4,{8F1B26BE-279A-53DB-E053-6C04A8C03649},321111,2006-12-20 00:00,SY8 1DJ,S,N,F,112,NaN,CORVE STREET,NaN,LUDLOW,SHROPSHIRE,SHROPSHIRE,A,A


In [44]:
%%time
### Data sources available
## model_data    ### low volume of various years
## model_data4   ### 2019 - 2019
## model_data5   ### 2018 - part 2
## model_data6   ### 2018 - part 1
## model_data7   ### 2017- part 2
## model_data8)  ### 2017 - part 1

### Concat files 
model_dataLOWVOLUME = model_data
model_2018 = pd.concat([model_data6, model_data5], ignore_index=True)
###model_2017 = pd.concat([model_data8, model_data7], ignore_index=True) ## this may be too big!?
model_2019 = model_data4
model_2017 = model_data8

CPU times: user 12.1 ms, sys: 50.4 ms, total: 62.5 ms
Wall time: 33.4 ms


In [56]:
len(model_dataLOWVOLUME)

100975

In [41]:
len(model_data8)

530864

In [42]:
len(model_2017)

1061729

In [43]:
len(model_data7)

530865

In [45]:
%%time
### train test validate
### date 2017

model_data_test = model_2017[model_2017['date'] == 2017] ##### used latter
model_data_train = model_data[model_data['date'] <= 2019] 
model_data_train2 = model_2019[model_2019['date'] == 2019] 
model_data_test = model_dataLOWVOLUME

CPU times: user 30.5 ms, sys: 21 ms, total: 51.5 ms
Wall time: 50.7 ms


In [58]:
len(model_data_test)

100975

In [27]:
model_data_train2.head(1)

,price,date,location_london,lease_encoded,type_D,type_F,type_O,type_S,type_T
0,100000,2019,0,1,0.0,1.0,0.0,0.0,0.0


In [ ]:
### SETTING UP TRAINING

In [ ]:
### data download and cleaning completed
### have a look
!ls house_price_data/
### next prepare the data for training

In [28]:
%%time
### TRAIN DATA TO CSV
model_data_train2.to_csv('train.csv', index=False, header=False)

CPU times: user 3.78 s, sys: 5.31 ms, total: 3.78 s
Wall time: 3.79 s


In [29]:
%%time
### S3 BUCKET
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

CPU times: user 173 ms, sys: 18.9 ms, total: 192 ms
Wall time: 502 ms


In [30]:
%%time
### SAGEMAKER
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

CPU times: user 22 µs, sys: 1 µs, total: 23 µs
Wall time: 27.2 µs


In [31]:
%%time
### SAGEMAKER SESSION
sess = sagemaker.Session()

CPU times: user 39.5 ms, sys: 4.03 ms, total: 43.5 ms
Wall time: 51.6 ms


In [32]:
%%time
### SAGEMAKER INSTANCE ----> reg:linear binary:logistic reg:squarederror
xgb = sagemaker.estimator.Estimator(containers[my_region],role, train_instance_count=1, train_instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)


CPU times: user 53 µs, sys: 3 µs, total: 56 µs
Wall time: 62.2 µs


In [33]:
%%time
### SET UP
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='reg:linear',num_round=1000)

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 15 µs


In [34]:
%%time
#### TRAIN
### may need to put validation in too validation parameter   ,"validation": validation file
xgb.fit({'train': s3_input_train})

2019-09-02 14:16:19 Starting - Starting the training job...
2019-09-02 14:16:22 Starting - Launching requested ML instances...
2019-09-02 14:17:18 Starting - Preparing the instances for training......
2019-09-02 14:18:06 Downloading - Downloading input data...
2019-09-02 14:18:44 Training - Training image download completed. Training in progress..
Arguments: train
[2019-09-02:14:18:45:INFO] Running standalone xgboost training.
[2019-09-02:14:18:45:INFO] Path /opt/ml/input/data/validation does not exist!
[2019-09-02:14:18:45:INFO] File size need to be processed in the node: 15.21mb. Available memory size in the node: 8606.52mb
[2019-09-02:14:18:45:INFO] Determined delimiter of CSV input is ','
[14:18:45] S3DistributionType set as FullyReplicated
[14:18:45] 444116x8 matrix with 3552928 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[14:18:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[0]#011tra

In [1]:
%%time
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

NameError: name 'xgb' is not defined

In [62]:
len(model_data_test)

100975

In [60]:
%%time
### LOAD DATA INTO AN ARRAY
test_data_array = model_data_test.drop(['price'], axis=1).values.astype(int) #load the data into an array

CPU times: user 5.15 ms, sys: 47 µs, total: 5.2 ms
Wall time: 4.21 ms


In [61]:
test_data_array

array([[2019,    0,    0, ...,    0,    0,    0],
       [2019,    0,    0, ...,    0,    1,    0],
       [2019,    0,    0, ...,    0,    1,    0],
       ...,
       [1995,    1,    0, ...,    0,    0,    1],
       [1995,    0,    0, ...,    0,    0,    0],
       [1995,    0,    1, ...,    0,    0,    0]])

In [63]:
%%time
### TEST PREDICTION
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [64]:
len(test_data_array)

100975

In [65]:
%%time
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict! WOW worked om lower volume

CPU times: user 1.17 s, sys: 1.98 ms, total: 1.17 s
Wall time: 7.49 s


In [67]:
len(predictions)

1267185

In [68]:
%%time
predictions_array = np.fromstring(predictions[1:], sep=',').astype(int) # and turn the prediction into an array
print(predictions_array.shape)

(100975,)
CPU times: user 21.6 ms, sys: 40 µs, total: 21.7 ms
Wall time: 20.8 ms


In [148]:
%%time
### normalised
### cm = pd.crosstab(index=round(model_data_test['price'],-5), columns=np.round(predictions_array,-5), rownames=['Observed'], colnames=['Predicted'],  normalize='index')
### non nornalised
cm = pd.crosstab(index=round(model_data_test['price'],-3), columns=np.round(predictions_array,-3), rownames=['Test'], colnames=['Predicted'])


CPU times: user 140 ms, sys: 3.57 ms, total: 143 ms
Wall time: 74.8 ms


In [149]:
cm

Predicted,94000,134000,180000,183000,188000,190000,231000,291000,394000,524000,...,563000,650000,654000,666000,795000,816000,917000,1077000,2698000,3755000
Test,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,33,0,0,2,2
1000,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,79,0,0,13,5
2000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,55,0,0,8,1
3000,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,34,0,0,3,0
4000,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,46,0,0,1,4
5000,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,25,0,0,4,1
6000,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,39,0,0,4,2
7000,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,13,0,0,1,1
8000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,19,0,0,1,1


In [84]:
from sklearn.metrics import accuracy_score

xx  = round(model_data_test['price'],-5)
yy = np.round(predictions_array,-5)

accuracy = accuracy_score(xx, yy)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 30.12%


In [ ]:
### get rid of everything
### clean up
### remove bucket contents

In [128]:
%%time
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)


CPU times: user 127 ms, sys: 3.89 ms, total: 131 ms
Wall time: 201 ms


In [ ]:
%%time
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)

In [ ]:
%%time
bucket_to_delete.objects.all().delete()

In [ ]:
### end

In [ ]:
### TESTING

In [90]:
len(model_data_test)

100975

In [101]:
#pd.DataFrame(predictions_array)

In [127]:
model_data_test.groupby('date')['price'].sum()

date
1995        2720000
1996        5506000
1997        7441000
1998        8606000
1999       10180000
2000        9722000
2001       12663000
2002       20043000
2003       15340000
2004       23693000
2005       14299000
2006       22676000
2007       32819000
2008       12717000
2009       11291000
2010       12356000
2011       13094000
2012       15534000
2013       26933000
2014       35405000
2015       51672000
2016      202961000
2017      383153000
2018     1915178000
2019    31443128000
Name: price, dtype: int64

In [147]:
model_data_test

,predicted,price,date,location_london,lease_encoded,type_D,type_F,type_O,type_S,type_T
50487,93791,571000,2019,0,0,1.0,0.0,0.0,0.0,0.0
67324,231464,135000,2019,0,0,0.0,0.0,0.0,1.0,0.0
67322,231464,183000,2019,0,0,0.0,0.0,0.0,1.0,0.0
67321,231464,190000,2019,0,0,0.0,0.0,0.0,1.0,0.0
67320,393791,375000,2019,0,0,1.0,0.0,0.0,0.0,0.0
67319,393791,375000,2019,0,0,1.0,0.0,0.0,0.0,0.0
67318,190275,94000,2019,0,0,0.0,0.0,0.0,0.0,1.0
67317,231464,182000,2019,0,0,0.0,0.0,0.0,1.0,0.0
67316,231464,220000,2019,0,0,0.0,0.0,0.0,1.0,0.0
67315,393791,240000,2019,0,0,1.0,0.0,0.0,0.0,0.0


In [89]:
len(pd.DataFrame(predictions_array))

100975

In [92]:
pd.DataFrame(predictions_array)

,0
0,93791
1,231464


In [ ]:
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn.linear_model import LinearRegression

In [ ]:

data = pd.read_csv('data.csv')  # load data set
X = data.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
Y = data.iloc[:, 1].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)  # make predictions

In [ ]:
plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.show()